In this notebook, I will experiment with different schedulers, seeing which can improve the standard coursework 1 setup.

In [ ]:
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=100, max_num_batches=1000, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

In [ ]:
#Baseline experiment

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
learning_rate = 0.5
max_epochs = 30
cost = CECost()
    
stats = []
for layer in xrange(1, 2):

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    # training_size should equal batch size, as that is the amount for each epoch
    lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

In [ ]:
#Baseline experiment

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateNewBob

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
learning_rate = 0.8
max_epochs = 30
cost = CECost()
    
stats = []
for layer in xrange(1, 2):

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                      min_derror_stop=.05, scale_by=0.05, zero_rate=0.5, patience = 10)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

In [ ]:
#Baseline experiment
%autoreload
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateNewBob, LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 600
learning_rate = 0.05
max_epochs = 10
cost = CECost()
    
stats = []
layer=2

train_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(Sigmoid(idim=784, odim=600, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=600, odim=500, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=500, odim=300, irange=0.2, rng=rng))
model.add_layer(Softmax(idim=300, odim=10, rng=rng))

lr_scheduler = LearningRateFixed(learning_rate=0.05, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Pre-Training started...')
tr_stats, valid_stats = optimiser.pretrain(model, train_dp, None, 0)
logger.info('Training started...')

train_dp.reset()

tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

In [ ]:
#Baseline experiment
%autoreload
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateNewBob, LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 600
learning_rate = 0.05
max_epochs = 10
cost = CECost()
    
stats = []
layer=2

train_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(Sigmoid(idim=784, odim=600, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=600, odim=500, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=500, odim=300, irange=0.2, rng=rng))
model.add_layer(Softmax(idim=300, odim=10, rng=rng))

lr_scheduler = LearningRateFixed(learning_rate=0.05, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')

train_dp.reset()

tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

In [ ]:
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
i = 0
inputs=[]

for x,t in train_dp:
    inputs.append(x)
    
print inputs[0].shape
print len(inputs)

In [ ]:
print len(inputs)

In [ ]:
#Baseline experiment
%autoreload
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateNewBob, LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 600
learning_rate = 0.05
max_epochs = 10
cost = CECost()
    
stats = []
layer=2

train_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(Sigmoid(idim=784, odim=600, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=600, odim=500, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=500, odim=300, irange=0.2, rng=rng))
model.add_layer(Softmax(idim=300, odim=10, rng=rng))

lr_scheduler = LearningRateFixed(learning_rate=0.05, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Pre-Training started...')
tr_stats, valid_stats = optimiser.pretrain_discriminative(model, train_dp, None)
logger.info('Training started...')

train_dp.reset()


tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

In [ ]:
#Run experiments using fixed, list, newBob and exponential use different scheduler each loop

In [1]:
# %load Experiments/scheduler.py
#Baseline experiment

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=100, max_num_batches=1000, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 5
cost = CECost()

#Open file to save to
shelve_p = shelve.open("learningRateExperiments")

stats = []
#Go through for each learning rate
for rate in xrange(1, 5):

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))
    
    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=0.5, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=0.5, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=0.5, max_epochs=max_epochs,\
                                      min_derror_stop=.05, scale_by=0.05, zero_rate=0.5, patience = 10)
    elif rate == 4:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateList([0.5,0.45,0.4,0.35,0.3,0.25,0.2,0.15,0.1,0.05,0.005],max_epochs)
    
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))
    
    if rate == 1:
        shelve_p['exponential'] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))
    elif rate == 2:
        shelve_p['fixed'] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))
    elif rate == 3:
        shelve_p['newbob'] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))
    elif rate == 4:
        shelve_p['list'] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))
        
        
shelve_p.close()   



%%writefile scheduler.py

SyntaxError: invalid syntax (<ipython-input-1-a31c96265b52>, line 86)

In [ ]:
Use 20 epochs, 0.01, 0.05, 0.1, 0.2, 0.5 
Always start from 0.5
Use both 1 and 2 layers and 3 layers as interested in different schedulers affects on depth of network.


